# <b>Price Prediction using XGBOOST

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import yfinance as yf

In [2]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S%26P_500_component_stocks')
table_symbol = data[0]
table_industry = data[0]['GICS Sector']

#these 2 lists are in order with one another
symbols = list(table_symbol.Symbol.values)
industries = list(table_industry.values) ##Industry Sector

remove = []

symbols_to_remove = ['BF.B', 'BRK.B', 'KVUE', 'VLTO','SPY']

for i, symbol in enumerate(symbols):
    if symbol in symbols_to_remove:
        remove.append(i)

for i in remove:
    symbols.remove(symbols[i])
    industries.remove(industries[i])

print(len(symbols))

symbol_industry = dict(zip(symbols,industries))
print(len(symbol_industry))

499
499


In [3]:
tickers = yf.Tickers('AAPL')

APPLdf = tickers.tickers['AAPL'].history(period="5d", start="2022-01-01", end="2022-12-31")
APPLdf.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,175.821100,180.814055,175.702461,179.953873,104487900,0.0,0.0
2022-01-04 00:00:00-05:00,180.566907,180.873403,177.096548,177.669998,99310400,0.0,0.0
2022-01-05 00:00:00-05:00,177.581006,178.134677,172.667149,172.943985,94537600,0.0,0.0
2022-01-06 00:00:00-05:00,170.749050,173.319685,169.701027,170.056961,96904000,0.0,0.0
2022-01-07 00:00:00-05:00,170.936919,172.172799,169.097931,170.225052,86709100,0.0,0.0
2022-01-10 00:00:00-05:00,167.169959,170.551323,166.270236,170.244827,106765600,0.0,0.0
2022-01-11 00:00:00-05:00,170.373372,173.201049,168.890317,173.102188,76138300,0.0,0.0
2022-01-12 00:00:00-05:00,174.130435,175.178458,172.845133,173.547104,74805200,0.0,0.0
2022-01-13 00:00:00-05:00,173.794269,174.624776,169.849337,170.244827,84505800,0.0,0.0


In [4]:
drop_cols = ['Open', 'Low', 'High','Volume', 'Dividends', 'Stock Splits']
APPLdf = APPLdf.drop(columns=drop_cols, axis = 1)
APPLdf.index = APPLdf.index.astype(str).str.split(' ').str[0]
APPLdf

,Close
Date,
2022-01-03,179.953873
2022-01-04,177.669998
2022-01-05,172.943985
2022-01-06,170.056961
2022-01-07,170.225052
...,...
2022-12-23,131.127060
2022-12-27,129.307236
2022-12-28,125.339417


In [5]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=1, subplot_titles=('Price of APPL for 2022',))

# Add trace for Close price
fig.add_trace(go.Scatter(x=APPLdf.index, y=APPLdf['Close'], name='Close Price'), row=1, col=1)

# Get unique months in the DataFrame
unique_months = pd.to_datetime(APPLdf.index).to_period('M').unique()

# Update x-axis layout to show ticks for each month
fig.update_xaxes(
    tickmode='array',
    tickvals=unique_months.to_timestamp(),  # Convert to timestamp for plotting
    ticktext=unique_months.strftime('%b'),  # Displaying abbreviated month names
)
# Show figure
fig.show()

In [6]:
holidays = ["2022-01-17","2022-02-21","2022-04-15", "2022-05-30", "2022-06-20", "2022-07-4", "2022-09-5", "2022-11-24","2022-12-26"]
len(holidays)

9

In [7]:
holiday_df = pd.DataFrame(index=holidays)
holiday_df['Date'] = holiday_df.index

# Add another column 'Close' with NaN values
holiday_df['Close'] = float('nan')

# Set the 'Date' column as the index
holiday_df.set_index('Date', inplace=True)

# Print the resulting DataFrame
holiday_df

,Close
Date,
2022-01-17,NaN
2022-02-21,NaN
2022-04-15,NaN
2022-05-30,NaN
2022-06-20,NaN
2022-07-4,NaN
2022-09-5,NaN
2022-11-24,NaN
2022-12-26,NaN


In [8]:
APPLdf.loc["2022-11-25"]

Close    147.286743
Name: 2022-11-25, dtype: float64

In [9]:
# Assuming APPLdf and holiday_df are DataFrames

# Concatenate the original DataFrame with the holiday DataFrame
frames = [APPLdf, holiday_df]
result_df = pd.concat(frames)

# Convert the index to a consistent data type (e.g., Timestamp)
result_df.index = pd.to_datetime(result_df.index)

# Sort the index to maintain order
result_df = result_df.sort_index()

# Print the resulting DataFrame
APPL1 = result_df
APPL1


,Close
Date,
2022-01-03,179.953873
2022-01-04,177.669998
2022-01-05,172.943985
2022-01-06,170.056961
2022-01-07,170.225052
...,...
2022-12-26,NaN
2022-12-27,129.307236
2022-12-28,125.339417


In [10]:
# Convert index to datetime if it's in string format
APPL1.index = pd.to_datetime(APPL1.index)

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame(index=[f'APPL Week {week}' for week in range(1, 53)],
                      columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'])

# Iterate through rows of the original DataFrame and fill the new DataFrame
for i, row in APPL1.iterrows():
    week = i.week
    day = i.dayofweek
    
    # If there are rows for the current week and day, set the mean close price
    if not pd.isna(row['Close']):
        new_df.at[f'APPL Week {week}', f'Day {day + 1}'] = row['Close']

# Display the new DataFrame
APPL = new_df
nan_rows = new_df[new_df.isna().any(axis=1)]
nan_rows

,Day 1,Day 2,Day 3,Day 4,Day 5
APPL Week 3,NaN,167.881836,164.352158,162.651581,160.575317
APPL Week 8,NaN,162.670731,158.463409,161.106598,163.195419
APPL Week 15,164.08638,165.977188,168.689713,163.630981,NaN
APPL Week 22,NaN,147.562592,147.433685,149.912262,144.132294
APPL Week 25,NaN,134.703888,134.188354,137.083313,140.444199
APPL Week 27,NaN,140.345062,141.693405,145.093964,145.778015
APPL Week 36,NaN,153.41655,154.836243,153.347061,156.236084
APPL Week 47,147.187302,149.345215,150.230286,NaN,147.286743
APPL Week 52,NaN,129.307236,125.339417,128.889572,129.207794


In [11]:
# Convert index to datetime if it's in string format
APPL1.index = pd.to_datetime(APPL1.index)

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame(index=[f'APPL Week {week}' for week in range(1, 53)],
                      columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'])

# Iterate through rows of the original DataFrame and fill the new DataFrame
for i, row in APPL1.iterrows():
    week = i.week
    day = i.dayofweek
    
    # If there are rows for the current week and day, set the mean close price
    if not pd.isna(row['Close']):
        new_df.at[f'APPL Week {week}', f'Day {day + 1}'] = row['Close']

# Drop rows with NaN values
new_df = new_df.dropna()

# Display the new DataFrame
APPL = new_df
APPL = APPL.drop(['APPL Week 1'], axis=0)
APPL


,Day 1,Day 2,Day 3,Day 4,Day 5
APPL Week 2,170.244827,173.102188,173.547104,170.244827,171.114899
APPL Week 4,159.79422,157.975006,157.886047,157.421356,168.405823
APPL Week 5,172.805542,172.637512,173.853592,170.946793,170.659729
APPL Week 6,169.937042,173.075241,174.510681,170.39241,166.947357
APPL Week 7,167.184952,171.05571,170.81813,167.184952,165.620804
APPL Week 9,163.462692,161.561951,164.888229,164.561539,161.532257
APPL Week 10,157.701111,155.859787,161.314468,156.92894,153.176987
APPL Week 11,149.108246,153.533356,157.98819,159.007874,162.334122
APPL Week 12,163.720093,167.125549,168.501617,172.322861,172.966339
APPL Week 13,173.837524,177.163773,175.985733,172.857452,172.560455


In [12]:
APPLPCT1 = APPL1.pct_change()
APPLPCT1

/var/folders/bq/q6bbfq_s68l_m1x6yxgvqnt80000gn/T/ipykernel_4024/1966763988.py:1: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.



,Close
Date,
2022-01-03,NaN
2022-01-04,-0.012691
2022-01-05,-0.026600
2022-01-06,-0.016693
2022-01-07,0.000988
...,...
2022-12-26,0.000000
2022-12-27,-0.013878
2022-12-28,-0.030685


In [13]:
# Convert index to datetime if it's in string format
APPLPCT1.index = pd.to_datetime(APPL1.index)

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame(index=[f'APPL PC Week {week}' for week in range(1, 53)],
                      columns=['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5'])

# Iterate through rows of the original DataFrame and fill the new DataFrame
for i, row in APPLPCT1.iterrows():
    week = i.week
    day = i.dayofweek
    
    # If there are rows for the current week and day, set the mean close price
    if not pd.isna(row['Close']):
        new_df.at[f'APPL PC Week {week}', f'Day {day + 1}'] = row['Close']

new_df.replace(0, pd.NA, inplace=True)

# Drop rows with NaN values
new_df = new_df.dropna()

# Display the new DataFrame
APPLPCT = new_df
APPLPCT

,Day 1,Day 2,Day 3,Day 4,Day 5
APPL PC Week 2,0.000116,0.016784,0.002570,-0.019028,0.005111
APPL PC Week 4,-0.004864,-0.011385,-0.000563,-0.002943,0.069777
APPL PC Week 5,0.026126,-0.000972,0.007044,-0.01672,-0.001679
APPL PC Week 6,-0.004235,0.018467,0.008294,-0.023599,-0.020218
APPL PC Week 7,0.001423,0.023153,-0.001389,-0.021269,-0.009356
APPL PC Week 9,0.001638,-0.011628,0.020588,-0.001981,-0.018408
APPL PC Week 10,-0.023718,-0.011676,0.034997,-0.027186,-0.023909
APPL PC Week 11,-0.026562,0.029677,0.029015,0.006454,0.020919
APPL PC Week 12,0.008538,0.020800,0.008234,0.022678,0.003734
APPL PC Week 13,0.005037,0.019134,-0.006649,-0.017776,-0.001718


In [14]:
tickers = yf.Tickers('MMM')

MMMdf = tickers.tickers['MMM'].history(period="5d", start="2022-01-01", end="2022-12-31")
drop_cols = ['Open', 'Low', 'High','Volume', 'Dividends', 'Stock Splits']
MMMdf = MMMdf.drop(columns=drop_cols, axis = 1)
MMMdf.index = MMMdf.index.astype(str).str.split(' ').str[0]
frames = [MMMdf, holiday_df]
result_df = pd.concat(frames)

# Convert the index to a consistent data type (e.g., Timestamp)
#result_df.index = pd.to_datetime(result_df.index)

# Sort the index to maintain order
result_df = result_df.sort_index()

# Print the resulting DataFrame
MMM1 = result_df
MMM1
pct = MMM1.ffill().pct_change()
pct

,Close
Date,
2022-01-03,NaN
2022-01-04,0.014009
2022-01-05,-0.004106
2022-01-06,-0.008301
2022-01-07,0.010955
...,...
2022-12-26,0.000000
2022-12-27,0.000666
2022-12-28,-0.016054


In [15]:
pct = MMM1.ffill().pct_change()
pct

,Close
Date,
2022-01-03,NaN
2022-01-04,0.014009
2022-01-05,-0.004106
2022-01-06,-0.008301
2022-01-07,0.010955
...,...
2022-12-26,0.000000
2022-12-27,0.000666
2022-12-28,-0.016054


In [16]:
# Get the list of S&P 500 stocks
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_tickers = sp500['Symbol'].tolist()
#removal = ['BF.B', 'KVUE', 'VLTO', 'BRK.B']

# Remove specified tickers from the list
#sp500_tickers = [ticker for ticker in sp500_tickers if ticker not in removal]
# Function to fetch data and create price percentage DataFrame
def get_price_percentage_df(ticker):
    try:
        stock_data = yf.download(ticker, start="2022-01-01", end="2022-12-31")
        drop_cols = ['Open', 'Low', 'High', 'Adj Close', 'Volume']
        stock_data = stock_data.drop(columns=drop_cols, axis=1)
        stock_data.index = stock_data.index.astype(str).str.split(' ').str[0]
        result_df = stock_data.ffill().pct_change()
        result_df.columns = [f"{ticker}" for col in result_df.columns]
        return result_df
    
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

In [17]:

# Fetch data for each ticker and create price percentage DataFrame
pct_dfs = []
for ticker in sp500_tickers:
        pct_df = get_price_percentage_df(ticker)
        if not pct_df.empty:
            pct_dfs.append(pct_df)

# Concatenate the DataFrames
sp500_pct_df = pd.concat(pct_dfs, axis=1)

# Display the resulting DataFrame
sp500_pct_df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******


1 Failed download:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-01-01 -> 2022-12-31)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******


1 Failed download:
['KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1672462800")



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******


1 Failed download:
['VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1672462800")



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

,MMM,AOS,ABT,ABBV,ACN,ADM,ADBE,ADP,AES,AFL,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,0.014009,0.016633,-0.023518,-0.001920,-0.007146,0.018559,-0.018374,0.004672,0.002871,0.032778,...,0.013160,0.020129,-0.003190,0.006329,0.010608,0.005347,0.006337,0.010145,0.028487,-0.038072
2022-01-05,-0.004106,-0.016714,-0.004493,0.005253,-0.017611,-0.007809,-0.071426,-0.008525,-0.045808,-0.001662,...,-0.006955,-0.023287,-0.037033,0.009800,-0.013629,-0.012677,-0.050068,-0.003297,0.002270,-0.038024
2022-01-06,-0.008301,0.006703,-0.000148,-0.004710,-0.048290,0.008891,-0.000603,-0.006871,-0.003858,0.018475,...,-0.011265,0.003502,0.004629,-0.007822,-0.001373,0.010331,-0.005410,-0.008307,0.041075,0.003971
2022-01-07,0.010955,-0.011296,0.003108,-0.002588,-0.019180,0.008523,-0.006652,-0.010564,-0.006024,0.015852,...,-0.011863,-0.008802,-0.001063,0.008759,-0.007992,-0.011466,-0.043771,-0.006050,0.016681,-0.029114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,-0.011925,0.011915,0.001389,-0.001041,0.005023,0.012087,0.005735,0.007443,0.013070,0.011846,...,0.008626,0.006644,-0.006401,0.012852,-0.000728,0.000621,0.002869,-0.000789,0.003521,0.005033
2022-12-27,0.000666,0.013532,0.003605,-0.000674,-0.002931,0.013633,-0.009928,0.000374,0.004881,0.006829,...,0.000777,0.000407,0.044723,0.009023,0.009022,0.007758,0.011200,0.004657,0.008050,-0.003156
2022-12-28,-0.016054,-0.015606,-0.006816,-0.004663,-0.008255,-0.023877,-0.020174,-0.013193,-0.017349,-0.010659,...,-0.011612,-0.014629,-0.050279,-0.013972,-0.016077,-0.004542,-0.016574,-0.010135,-0.017813,-0.010117


In [18]:
sp500_pct_df.T
sp500_pct_df.T.select_dtypes(include='object')

Date
MMM
AOS
ABT
ABBV
ACN
...
YUM
ZBRA
ZBH
ZION


In [19]:
import pandas as pd

holidays = ["2022-01-17", "2022-02-21", "2022-04-15", "2022-05-30", "2022-06-20", "2022-07-4", "2022-09-5", "2022-11-24", "2022-12-26"]
columns = sp500_pct_df.columns

# Create holiday_df with NaN values for each date and columns matching sp500_pct_df
holiday_df = pd.DataFrame(index=holidays, columns=columns)
holiday_df['Date'] = holiday_df.index

# Set the 'Date' column as the index
holiday_df.set_index('Date', inplace=True)

# Convert all values to float64
holiday_df = holiday_df.astype(float)

# Display the shape of the DataFrame
holiday_df.shape
holiday_df.dtypes


MMM     float64
AOS     float64
ABT     float64
ABBV    float64
ACN     float64
         ...   
YUM     float64
ZBRA    float64
ZBH     float64
ZION    float64
ZTS     float64
Length: 499, dtype: object

In [20]:
result_df = pd.concat([sp500_pct_df, holiday_df], axis=0)
# Sort the DataFrame by index to maintain order
result_df = result_df.sort_index()
sp500_pct_df = result_df

In [21]:
sp500_pct_df.select_dtypes(include='object')

""
Date
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
...
2022-12-26
2022-12-27
2022-12-28


In [22]:
sp500_pct_df.T

Date,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-10,2022-01-11,2022-01-12,2022-01-13,2022-01-14,...,2022-12-19,2022-12-20,2022-12-21,2022-12-22,2022-12-23,2022-12-26,2022-12-27,2022-12-28,2022-12-29,2022-12-30
MMM,NaN,0.014009,-0.004106,-0.008301,0.010955,-0.014115,0.008850,0.002458,0.008918,-0.012595,...,0.003616,-0.010808,0.021935,-0.015147,-0.011925,NaN,0.000666,-0.016054,0.019275,-0.005391
AOS,NaN,0.016633,-0.016714,0.006703,-0.011296,-0.003127,0.009772,0.003345,-0.012384,-0.028575,...,0.000178,0.004995,0.015974,-0.017645,0.011915,NaN,0.013532,-0.015606,0.015677,-0.007284
ABT,NaN,-0.023518,-0.004493,-0.000148,0.003108,-0.002213,0.003253,-0.014591,-0.030437,-0.011492,...,-0.002993,-0.000938,0.015494,-0.001017,0.001389,NaN,0.003605,-0.006816,0.022999,-0.004714
ABBV,NaN,-0.001920,0.005253,-0.004710,-0.002588,0.011195,0.004253,0.002336,-0.027460,0.017600,...,0.006294,-0.005635,0.010151,0.006535,-0.001041,NaN,-0.000674,-0.004663,0.002034,-0.005844
ACN,NaN,-0.007146,-0.017611,-0.048290,-0.019180,0.006069,0.005657,0.005545,-0.041942,-0.022166,...,-0.034332,0.025450,0.026804,-0.015469,0.005023,NaN,-0.002931,-0.008255,0.019991,-0.005738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,0.005347,-0.012677,0.010331,-0.011466,-0.013964,-0.020980,-0.011557,-0.002633,-0.013586,...,-0.007652,0.001636,0.001478,0.000311,0.000621,NaN,0.007758,-0.004542,0.005259,-0.014693
ZBRA,NaN,0.006337,-0.050068,-0.005410,-0.043771,0.008571,0.005902,-0.000241,-0.023475,0.004184,...,-0.020930,0.014115,0.014930,-0.013275,0.002869,NaN,0.011200,-0.016574,0.043307,-0.004349
ZBH,NaN,0.010145,-0.003297,-0.008307,-0.006050,-0.009754,0.011663,-0.021810,0.019828,-0.005388,...,-0.004937,0.003761,0.013155,-0.002282,-0.000789,NaN,0.004657,-0.010135,0.014604,-0.002582
ZION,NaN,0.028487,0.002270,0.041075,0.016681,-0.007562,0.009488,-0.000142,-0.002136,0.008421,...,-0.007163,0.008526,0.031216,0.014925,0.003521,NaN,0.008050,-0.017813,0.023139,0.001630


In [23]:
holiday_df.dtypes

MMM     float64
AOS     float64
ABT     float64
ABBV    float64
ACN     float64
         ...   
YUM     float64
ZBRA    float64
ZBH     float64
ZION    float64
ZTS     float64
Length: 499, dtype: object

In [24]:
sp500_pct_df.dtypes

MMM     float64
AOS     float64
ABT     float64
ABBV    float64
ACN     float64
         ...   
YUM     float64
ZBRA    float64
ZBH     float64
ZION    float64
ZTS     float64
Length: 499, dtype: object

In [25]:
company_values = sp500_pct_df.T.iloc[0].values

# Create a list of week strings
weeks = [f"MMM Week {i}" for i in range(1, 53)]

# Create an empty DataFrame with the specified index and columns
MMMweeks = pd.DataFrame(index=weeks, columns=["Day 1", "Day 2", "Day 3", "Day 4", "Day 5"])

# Set the values in MMMweeks
for i in range(0, len(company_values), 5):
    week_number = (i // 5) + 1
    week_name = f"MMM Week {week_number}"
    MMMweeks.loc[week_name] = company_values[i:i + 5]

# Display the resulting DataFrame
MMMweeks.head()

,Day 1,Day 2,Day 3,Day 4,Day 5
MMM Week 1,NaN,0.014009,-0.004106,-0.008301,0.010955
MMM Week 2,-0.014115,0.00885,0.002458,0.008918,-0.012595
MMM Week 3,NaN,-0.002574,0.001122,-0.02751,-0.005301
MMM Week 4,0.000869,0.005498,-0.025611,0.00508,-0.04149
MMM Week 5,0.017903,0.008493,-0.004599,-0.013621,-0.022264


In [26]:
MMMdf.pct_change().head()

,Close
Date,
2022-01-03,NaN
2022-01-04,0.014009
2022-01-05,-0.004106
2022-01-06,-0.008301
2022-01-07,0.010955


In [27]:
MMMdf.dtypes

Close    float64
dtype: object

In [28]:
dfs = []

# Iterate through each stock
for stock in sp500_pct_df.T.index:
    # Get the values for the current stock
    stock_values = sp500_pct_df.T.loc[stock].values
    
    # Create a list of week strings
    weeks = [f"{stock} Week {i}" for i in range(1, 53)]

    # Create an empty DataFrame with the specified index and columns
    stock_df = pd.DataFrame(index=weeks, columns=["Day 1", "Day 2", "Day 3", "Day 4", "Day 5"])

    # Set the values in the DataFrame
    for i in range(0, len(stock_values), 5):
        week_number = (i // 5) + 1
        week_name = f"{stock} Week {week_number}"
        stock_df.loc[week_name] = stock_values[i:i + 5]

    # Append the DataFrame for the current stock to the list
    dfs.append(stock_df)

# Concatenate all DataFrames in the list
result_df = pd.concat(dfs)

# Drop rows with NaN values
result_df = result_df.dropna()

# Display the resulting DataFrame
sp500_pct_df = result_df
sp500_pct_df

,Day 1,Day 2,Day 3,Day 4,Day 5
MMM Week 2,-0.014115,0.00885,0.002458,0.008918,-0.012595
MMM Week 4,0.000869,0.005498,-0.025611,0.00508,-0.04149
MMM Week 5,0.017903,0.008493,-0.004599,-0.013621,-0.022264
MMM Week 6,0.002738,0.007694,0.007573,-0.023833,-0.001252
MMM Week 7,-0.00959,-0.00424,-0.010868,-0.044914,-0.006862
...,...,...,...,...,...
ZTS Week 46,-0.014137,0.023011,-0.010279,-0.031562,0.014206
ZTS Week 48,-0.013125,-0.00189,0.042543,0.016219,0.00498
ZTS Week 49,-0.013277,-0.014678,-0.018295,0.022829,-0.001887
ZTS Week 50,-0.003455,0.013803,-0.020585,-0.020227,-0.028579


In [29]:
dropweeks = [1,3,8,15,22,25,26,27,36,47,52]

In [30]:
sp500_pct_df = sp500_pct_df.apply(pd.to_numeric, errors='coerce')
sp500_pct_df.dtypes

Day 1    float64
Day 2    float64
Day 3    float64
Day 4    float64
Day 5    float64
dtype: object

# <b> XGBOOST MODEL

In [31]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def train_xgboost_regressor(X, y, params=None, test_size=0.2, random_state=42):
    """
    Train an XGBoost regressor on the given dataset.

    Parameters:
    - X: Features
    - y: Labels
    - params: XGBoost parameters (default is None, indicating to use default parameters)
    - test_size: Proportion of the dataset to include in the test split
    - random_state: Random seed for reproducibility

    Returns:
    - model: Trained XGBoost regressor
    - X_train, X_test, y_train, y_test: Train-test split of the dataset
    - y_pred: Predictions on the test set
    - mse: Mean squared error on the test set
    """

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Initialize the XGBoost regressor
    model = xgb.XGBRegressor() if params is None else xgb.XGBRegressor(**params)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate mean squared error
    mse = mean_squared_error(y_test, y_pred)

    return model, X_train, X_test, y_train, y_test, y_pred, mse

# Example usage:
# Assume X and y are your feature matrix and target variable
# model, X_train, X_test, y_train, y_test, y_pred, mse = train_xgboost_regressor(X, y)


In [32]:
y = sp500_pct_df['Day 5']
X = sp500_pct_df.drop(columns='Day 5')

In [33]:
sp500_pct_df = sp500_pct_df.astype('float64')

In [34]:
train_xgboost_regressor(X,y)

(XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...),
                  Day 1     Day 2     Day 3     Day 4
 URI Week 24  -0.054100 -0.018637  0.022451 -0.076778
 ORLY Week 17  0.007983  0.001163 -0.001344 -0.126190
 NXPI Week 18  0.035693  0.006045  0.044084 -0.040609
 ES Week 48   -0.008734 -